In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other


In [4]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [8]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [9]:
twitter_data.shape
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [11]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
twitter_data.shape

(1600000, 6)

In [13]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [14]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [15]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [16]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [17]:
port_stem=PorterStemmer()

In [18]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z ]','',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [19]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [20]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpiccomyzl awww that bummer ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cant updat facebook text might cri resul...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav im mad cant see


In [21]:
print(twitter_data['stemmed_content'])

0          switchfoot httptwitpiccomyzl awww that bummer ...
1          upset cant updat facebook text might cri resul...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                      nationwideclass behav im mad cant see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdbcom cool hear old walt interview httpbli...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [22]:
X=twitter_data['stemmed_content'].astype(str)
y=twitter_data['target'].values

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

In [24]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [25]:
print(X_train)


1570269                          watch saw iv drink lil wine
1273074                                      hatermagazin im
88479      even though favourit drink think vodka coke wi...
254604                   think hand got burnt sun today hurt
667941     took mazi drfor shot today come find ear infec...
                                 ...                        
941805                                       threewink cheer
1007131    vote livewir play live smith tomorrow night su...
1460311                               eager monday afternoon
929226     hope everyon mother great day cant wait hear g...
526253                      love wake folger bad voic deeper
Name: stemmed_content, Length: 1280000, dtype: object


In [26]:
print(X_test)

131348     mmangen fine havent much time chat twitter hub...
1142114         ah may show w ruth kim amp geoffrey sanhueza
244564                   ishatara mayb bay area thang dammit
445353                           game endedthey lost stinkyy
415893                                          cool brother
                                 ...                        
178459     twitter drive nutswont let download profil pic...
1515130                               teamqivana your welcom
1449952    destini nevertheless hooray member wonder safe...
441063                                             feel well
1583304                                    supersandro thank
Name: stemmed_content, Length: 320000, dtype: object


In [27]:
print(type(X_train))
print(X_train[:2])

<class 'pandas.core.series.Series'>
1570269    watch saw iv drink lil wine
1273074                hatermagazin im
Name: stemmed_content, dtype: object


In [28]:
#converting text data to numerical data
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [29]:
print(X_train)
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9508205 stored elements and shape (1280000, 622304)>
  Coords	Values
  (0, 590524)	0.271935973896376
  (0, 485829)	0.35734734000828555
  (0, 275153)	0.5287163126006141
  (0, 135672)	0.3751210798745192
  (0, 330512)	0.4200753866446622
  (0, 599548)	0.4490363130629349
  (1, 199672)	0.9736525845879415
  (1, 265990)	0.2280364982304841
  (2, 135672)	0.46222653304912487
  (2, 265990)	0.11789035957962253
  (2, 153180)	0.1905056115706456
  (2, 552451)	0.1883319097978993
  (2, 160229)	0.29161611256480313
  (2, 551043)	0.3225385174402054
  (2, 586742)	0.3320039094486021
  (2, 96518)	0.3154389006079266
  (2, 600077)	0.33687662733057383
  (2, 368923)	0.24369409966007974
  (2, 555076)	0.1531113773352699
  (2, 186397)	0.1889237729533197
  (2, 163730)	0.20357992328484822
  (2, 395952)	0.16862711364982916
  (3, 551043)	0.28972941659572027
  (3, 197116)	0.4472150500940576
  (3, 188008)	0.2779040561459071
  :	:
  (1279996, 587294)	0.271129357

In [30]:
#Training the machine learning model (Logistic Regression)
model=LogisticRegression(max_iter=1000)

In [31]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [32]:
#model evaluation
#accuracy score of training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(y_train,X_train_prediction)

In [33]:
print('accuracy score on the training data = ',training_data_accuracy)

accuracy score on the training data =  0.80700703125


In [34]:
#model evaluation
#accuracy score of test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(y_test,X_test_prediction)

In [35]:
print('model accuracy on test data',test_data_accuracy)

model accuracy on test data 0.781696875


In [41]:
new_tweet=['Today is a bad day.']
new_vector = vectorizer.transform(new_tweet)
prediction= model.predict(new_vector)

In [42]:
print('prediction',prediction)

prediction [0]


In [44]:
#Saving the trained model.
import pickle

In [45]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [46]:
#Using the saved model for future predictions
#loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [49]:
X_new=X_test[200]
print(X_test[200])
print(y_test[200]) #The 200th test tweet is a positive tweet with y value as 1


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 622304)>
  Coords	Values
  (0, 116397)	0.18895757337640626
  (0, 156909)	0.32979679667675
  (0, 180780)	0.18538117057836828
  (0, 339508)	0.20126428223596468
  (0, 445061)	0.3038750302123858
  (0, 509301)	0.3480772775903513
  (0, 528002)	0.4670092546086213
  (0, 543133)	0.3349087095329932
  (0, 576838)	0.4867199460511626
1


In [52]:
prediction=loaded_model.predict(X_new)
print(prediction)
if(prediction[0]==0):
  print('It is a negative tweet.')
else:
  print('It is a positive tweet.')

[1]
It is a positive tweet.
